In [5]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import re

# 크롬 드라이버 사용 가능하도록 드라이버 설치 및 전체화면 설정
driver = webdriver.Chrome(executable_path=ChromeDriverManager(version="114.0.5735.16").install())
driver.maximize_window()

# Youtube URL List(확장자: .csv) 읽어오기 
# 환경에 맞게 경로 변경 필요
input_data = pd.read_csv('D:/PJB/MyDatalab/Jupyter Notebook/Input-data/Youtube_Content_URL_list.csv')
url = input_data['URL']
url_val = url.values
url_list = url_val.tolist()

# 구독자 수 한글 표기 없이 숫자로만 표현 가능하게끔 하는 함수1
def convert_korean_to_number(korean_str):
    units = {'천': 1000, '만': 10000}
    number = float(re.search(r'[\d.]+', korean_str).group())
    unit_char = re.search(r'[천만]', korean_str)
    if unit_char:
        unit = units[unit_char.group()]
        number *= unit
    return int(number)

# 구독자 수 한글 표기 없이 숫자로만 표현 가능하게끔 하는 함수2
def extract_subscriber_count(input_str):
    pattern = r'구독자\s+([\d.천만]+)명'
    match = re.search(pattern, input_str)
    if match:
        subscriber_count = match.group(1)
        return convert_korean_to_number(subscriber_count)
    return None

# 필요한 데이터 리스트
title_list = []   # 제목
publisher_list = []   # 게시자
subscriber_list = []   # 구독자 수
date_list = []   # 게시일
like_list = []   # 좋아요 수
comment_list = []   # 댓글 수
view_list = []   # 조회수

#  읽어들인 url list에 있는 url 하나씩 실행하며 필요한 데이터 추출
for url in url_list[:]:
    driver.get(url)
    time.sleep(2)
    
    # 제목
    title = driver.find_element_by_css_selector('#title > h1')
    title_list.append(title.text)
    time.sleep(2)
    
    # 게시자
    publisher = driver.find_element_by_css_selector('#text > a')
    publisher_list.append(publisher.text)
    time.sleep(2)
    
    # 구독자 수
    subscriber = driver.find_element_by_css_selector('#owner-sub-count')
    subscriber_text = subscriber.text
    subscriber_int = extract_subscriber_count(subscriber_text)
    subscriber_list.append(subscriber_int)
    time.sleep(2)
    
    # 게시일
    driver.find_element_by_xpath('//*[@id="expand"]').click()
    date = driver.find_element_by_css_selector('#info > span:nth-child(3)')
    date_list.append(date.text)
    time.sleep(2)
    
    # 좋아요 수
    like = driver.find_element_by_xpath('//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button')
    like_text = like.get_attribute('aria-label')
    like_int = re.sub(r'[^0-9]', '', like_text)
    like_list.append(like_int)
    time.sleep(2)
    
    # 댓글 수
    driver.find_element_by_tag_name('html').send_keys(Keys.PAGE_DOWN)
    driver.find_element_by_tag_name('html').send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    try:
        comment = driver.find_element_by_css_selector('#count > yt-formatted-string')
        comment_text = comment.text
        comment_int = re.sub(r'[^0-9]', '', comment_text)
        comment_list.append(comment_int)
    # 댓글 사용 중지인 동영상의 경우 댓글 수 0개로 처리
    except:
        comment = '0'
        comment_list.append(comment)
    
    # 조회수
    view = driver.find_element_by_css_selector('#info > span:nth-child(1)')
    view_text = view.text
    view_int = re.sub(r'[^0-9]', '', view_text)
    view_list.append(view_int)
    time.sleep(2)

df = {'제목': title_list, 'URL': url_list, '게시자': publisher_list, '구독자 수': subscriber_list, '게시일': date_list, '좋아요 수': like_list, '댓글 수': comment_list, '조회수': view_list}
output_data = pd.DataFrame(df)
# 추출된 데이터 .csv 파일로 저장
# 환경에 맞게 저장 경로 변경 필요
output_data.to_csv("D:/PJB/MyDatalab/Develop-Crawling/Crawling_MyDatalab/Output-data/youtube_data/youtube_data.csv", index = False, encoding='utf-8-sig')



====== WebDriver manager ======
Trying to download new driver from https://chromedriver.storage.googleapis.com/114.0.5735.16/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\0226p\.wdm\drivers\chromedriver\win32\114.0.5735.16]
